# Day 10: Monitoring Station
https://adventofcode.com/2019/day/10

## Part 1

In [ ]:
import numpy as np
import urllib.request
import math
import itertools
import re

In [ ]:
def checkMapIsValid(string_map):
    #Map must have one line, at least
    if len(string_map) == 0:
        return False
    #First line must have columns
    width = len(string_map[0])
    if width == 0:
        return False
    
    #All lines must have same length
    for num_line in range(1, len(string_map)):
        if len(string_map[num_line]) != width:
            return False
    return True
    
def loadMap(string_map):
    assert checkMapIsValid(string_map), 'Map is not valid!'
    
    height = len(string_map)
    width  = len(string_map[0])
    
    #Initializing zero-matrix and filling with one when an
    #asteroid is found
    int_map = np.zeros([height, width])
    for num_line in range(height):
        line = string_map[num_line]
        for num_col in range(width):
            asteroid = line[num_col:num_col+1]
            if asteroid == '#':
                int_map[num_line][num_col] = 1
    
    return int_map

In [ ]:
string_map = ['..###.',
              '.#....']

In [ ]:
int_map = loadMap(string_map)
print(string_map)
print(int_map)

In [ ]:
def getModulus(x, y):
    return math.sqrt(math.pow(x, 2) + math.pow(y, 2))

def probeAsteroid(int_map, asteroid_col, asteroid_line):
    
    height = len(int_map)
    width  = len(int_map[0])
    
    #Starting point must be an asteroid...
    assert int_map[asteroid_line][asteroid_col] == 1, f'There is no asteroid on [{asteroid_col}, {asteroid_line}]!'
    
    # Using the probing asteroid as origin, get a vector to other asteroids, to get distance and direction.
    # We'll store all asteroids in a dictionary indexed by direction. The value of the dictionary will be
    # a list of all asteroids on the same direction sorted by distance. Only the first one (or the only one)
    # in this list will be visible.
    dict_vectores = {}
    for line in range(height):
        for col in range(width):
            #The asteroid is not visible to itself...
            if line == asteroid_line and col == asteroid_col:
                continue
            #Only asteroids, please...
            if int_map[line][col] == 0:
                continue
            norm_line = line - asteroid_line
            norm_col  = col - asteroid_col
            modulus = getModulus(norm_col, norm_line)
            #Rounding to 10 decimal positions to avoid floating point problems...
            norm_line = round(norm_line / modulus, 10)
            norm_col  = round(norm_col / modulus, 10)
            norm_coords = ( norm_col, norm_line )
            
            if not norm_coords in dict_vectores:
                dict_vectores[norm_coords] = []
            dict_vectores[norm_coords].append( ( modulus, ( col, line )  ) )

    # For convenience we create two other dictionaries, one with only the visible ones and the other one
    # with the no visible ones. This could have been made outside this function but oh, well..
    #Para cada dirección, nos quedamos con el que menos módulo tiene
    dict_visibles = {}
    dict_novisibles = {}
    for norm_coords in dict_vectores:
        candidates = dict_vectores[norm_coords]
        candidates.sort()
        (modulus, map_coords) = candidates[0]
        dict_visibles[map_coords] = modulus
        for otro_asteroide in candidates[1:]:
            (modulus, map_coords) = otro_asteroide
            dict_novisibles[map_coords] = candidates[0]
            
    return len(dict_visibles), dict_vectores, dict_visibles, dict_novisibles

In [ ]:
#A little convenience function to print results
def printResult(result, x, y):
    (num_visibles, dic, dic_vis, dic_novis) = result
    print(f'Visible asteroids from ({x},{y}):', num_visibles)
    print(f'All asteroids from ({x},{y}):', dic)
    print(f'Visible asteroids from ({x},{y}):', dic_vis)
    print(f'Non visible asteroids from ({x},{y}):', dic_novis)

In [ ]:
printResult(probeAsteroid(int_map, 2, 0), 2, 0)

In [ ]:
#Given a map, search the best location to locate the monitor station. We'll probe all asterorids to maximize
#the number of asteroids found. This can be improved:
# * Making a list of asteroids with coordinates. In very large and sparse maps should improve performance a lot.
# * Pass the number of asteroids already found to the probing function so if 'asteroids already found' +
#   'number of positions to probe yet' < 'maximum asteroids visible probing other asteroids' don't continue probing and
#   process the next asteroid, because you are not going to improve the count already found.
def searchBestLocation(int_map):
    height = len(int_map)
    width  = len(int_map[0])
    
    best_bet = -1
    best_asteroid = None

    for line in range(height):
        for col in range(width):
            if int_map[line][col] == 0:
                continue
            num_asteroids, _, _, _ = probeAsteroid(int_map, col, line)
            
            if num_asteroids > best_bet:
                best_bet = num_asteroids
                best_asteroid = (col, line)
                
    return best_asteroid, best_bet

In [ ]:
best_asteroid, num_asteroids = searchBestLocation(int_map)
print('Best location:', best_asteroid, 'Visible asteroids:', num_asteroids)

### Tests

#### Test 1:

Map:

```
.#..#
.....
#####
....#
...##
```

Best location is (3, 4), 8 visible asteroids

In [ ]:
string_map = ['.#..#',
            '.....',
            '#####',
            '....#',
            '...##']

int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

In [ ]:
printResult(probeAsteroid(int_map, 1, 0), 1, 0)

In [ ]:
printResult(probeAsteroid(int_map, 3, 4), 3,4)

#### Test 2:

Map:

```
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####
```

Best location is (5, 8), 33 visible asteroids

In [ ]:
string_map = ['......#.#.',
'#..#.#....',
'..#######.',
'.#.#.###..',
'.#..#.....',
'..#....#.#',
'#..#....#.',
'.##.#..###',
'##...#..#.',
'.#....####']


int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

#### Test 3:

Map:

```
#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.
```

Best location is (1, 2), 35 visible asteroids

In [ ]:
string_map = ['#.#...#.#.',
'.###....#.',
'.#....#...',
'##.#.#.#.#',
'....#.#.#.',
'.##..###.#',
'..#...##..',
'..##....##',
'......#...',
'.####.###.']

int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

#### Test 4:

Map:

```
.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..
```

Best location is (6, 3), 41 visible asteroids

In [ ]:
string_map = ['.#..#..###',
'####.###.#',
'....###.#.',
'..###.##.#',
'##.##.#.#.',
'....###..#',
'..#.#..#.#',
'#..#.#.###',
'.##...##.#',
'.....#.#..']


int_map = loadMap(string_map)
print(int_map)
searchBestLocation(int_map)

#### Test 5 (Big Example):

Map:

```
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
```

Best location is (11, 13), 210 visible asteroids

In [ ]:
big_example_string_map = ['.#..##.###...#######',
'##.############..##.',
'.#.######.########.#',
'.###.#######.####.#.',
'#####.##.#.##.###.##',
'..#####..#.#########',
'####################',
'#.####....###.#.#.##',
'##.#################',
'#####.##.###..####..',
'..######..##.#######',
'####.##.####...##..#',
'.#####..#.######.###',
'##...#.##########...',
'#.##########.#######',
'.####.#.###.###.#.##',
'....##.##.###..#####',
'.#.#.###########.###',
'#.#.#.#####.####.###',
'###.##.####.##.#..##']


big_example_int_map = loadMap(big_example_string_map)
print(big_example_int_map)
searchBestLocation(big_example_int_map)

### Solution

In [ ]:
input_10 = r'data\aoc2019-input-day10.txt'
with open(input_10, 'r') as f:
    data10 = [re.sub("\n", "", l) for l in f.readlines()]
data10

In [ ]:
int_map_data10 = loadMap(data10)
print(int_map_data10)
searchBestLocation(int_map_data10)

>>>SOLUTION: 253

## Part 2

In [ ]:
def getAngleInRads(x, y):
    #Base cases, both axes, both ways
    if x == 0:
        if y >= 0:
            return np.pi / 2
        elif y < 0:
            return 3 * np.pi / 2

    if y == 0:
        if x >= 0:
            return 0
        elif x < 0:
            return np.pi
    # vector (x, y)
    # tan PHI = | y / x |
    tang = np.abs( y / x )
    angle = math.atan(tang)
    
    #Adjusting angle by quadrant
    if x < 0 and y > 0: #Second quadrant
        return np.pi - angle
    elif x < 0 and y < 0:
        return np.pi + angle #Third quadrant
    elif x > 0 and y < 0:
        return (2* np.pi) - angle #Forth quadrant
    
    return angle #First quadrant

### Testing angle calculation

In [ ]:
print('First quadrant bisection:', 'calculated:', getAngleInRads(1,1), 'expected:', np.pi / 4)
print('Second quadrant bisection:', 'calculated:', getAngleInRads(-1,1), 'expected:', (3 * np.pi) / 4 )
print('Third quadrant bisection:', 'calculated:', getAngleInRads(-1,-1), 'expected:', (5 * np.pi) / 4 )
print('Fourth quadrant bisection:', 'calculated:', getAngleInRads(1,-1), 'expected:', (7 * np.pi) / 4 )

print('+y axis:', 'calculated:', getAngleInRads(0,1), 'expected:', np.pi / 2 )
print('-y axis:', 'calculated:', getAngleInRads(0,-1), 'expected:', (3 * np.pi) / 2 )
print('+x axis:', 'calculated:', getAngleInRads(1,0), 'expected:', 0 )
print('-x axis:', 'calculated:', getAngleInRads(-1,0), 'expected:', np.pi )

In [ ]:
def sortAsteroidsForDestructionGroupedByDirection(all_asteroids, sort_clockwise=True):
    newAngles = []
    for i in all_asteroids:
        x, y = i
        #Flip y axis to get same coordinates system
        y *= -1
        angleRads = getAngleInRads(x, y)

        #Because we want to start vaporizing asteroids in the y axis, we "turn" pi/2 rads the
        #map so, at 0 rads we will be exactly over the station
        newRads = angleRads - (np.pi / 2)
        #If the asteroid was in the first quadrant, we add an entire turn so when they are reverse sorted
        #by angle, they will be the first ones.
        if newRads <= 0:
            newRads += 2 * np.pi

        #building new items: transformed angle, vector, list of all asteroids in the same direction, 
        #ordered by how far of the station they are
        tup = ( newRads, i, all_asteroids[i])
        newAngles.append(tup)
    
    #angles are counterclockwise, so...
    newAngles.sort(reverse=sort_clockwise)
        
    return newAngles

### Testing asteroid sorting

#### Test 1:

* Map (with station at absolute coordinates (8,3)):
```
.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....X...###..
..#.#.....#....##
```

* Destruction order (from station point of view and map point of view):
    * First 9
        * (0, 2) -> ( 8, 1)
        * (1, 3) -> ( 9, 0)
        * (1, 2) -> ( 9, 1)
        * (2, 3) -> (10, 0)
        * (1, 1) -> ( 9, 2)
        * (3, 2) -> (11, 1)
        * (4, 2) -> (12, 1)
        * (3, 1) -> (11, 2)
        * (7, 2) -> (15, 1)
    * Next 9
        * ( 4, 1) -> (12, 2)
        * ( 5, 1) -> (13, 2)
        * ( 6, 1) -> (14, 2)
        * ( 7, 1) -> (15, 2)
        * ( 4, 0) -> (12, 3)
        * ( 8,-1) -> (16, 4)
        * ( 7,-1) -> (15, 4)
        * ( 2,-1) -> (10, 4)
        * (-4,-1) -> ( 4, 4)
    * Next 9
        * (-6,-1) -> ( 2, 4)
        * (-6, 0) -> ( 2, 3)
        * (-8, 1) -> ( 0, 2)
        * (-7, 1) -> ( 1, 2)
        * (-8, 2) -> ( 0, 1)
        * (-7, 2) -> ( 1, 1)
        * (-3, 1) -> ( 5, 2)
        * (-7, 3) -> ( 1, 0)
        * (-3, 2) -> ( 5, 1)
    * Last 9
        * (-2, 2) -> ( 6, 1)
        * (-2, 3) -> ( 6, 0)
        * (-1, 3) -> ( 7, 0)
        * ( 0, 3) -> ( 8, 0) [Second round]
        * ( 2, 2) -> (10, 1) [Second round]
        * ( 6, 3) -> (14, 0) [Second round]
        * ( 8, 2) -> (16, 1) [Second round]
        * ( 5, 0) -> (13, 3) [Second round]
        * ( 6, 0) -> (14, 3) [Second round]


In [ ]:
#Test map as string
string_map = ['.#....#####...#..',
              '##...##.#####..##',
              '##...#...#.#####.',
              '..#.....#...###..',
              '..#.#.....#....##']
#Numeric representation
int_map = loadMap(string_map)

#Given (8,3) as station location, obtaining visibility of all asteriods from there
_, all_asteroids_from_station, _, _ = probeAsteroid(int_map, 8, 3)

#Ordering asteroids for destruction by angle. Grouped by the same direction
sorted_asteroids = sortAsteroidsForDestructionGroupedByDirection(all_asteroids_from_station)

#Showing visible asteroids in first turn, in groups of 9, like the example
for i in range(len(sorted_asteroids)):
    print(sorted_asteroids[i])
    if (i + 1) % 9 == 0:
        print(20 * '--')


In [ ]:
def vaporizeAsteroids(clockwise_asteroids):
    asteroid_counter = 1
    asteroid_list = clockwise_asteroids
    # Each iteration we traverse all directions in the list and build a new list
    # with directions that have asteroids left. So, directions without any asteroid left are purged.
    # Iterations ends when no direction is left.
    while len(asteroid_list) > 0:
        new_list = []
        for i in asteroid_list:
            (angle, direction, asteroids) = i
            asteroid = asteroids[0]
            (range_from_station, asteroid_coords) = asteroid
            # Pew pew...
            print('Vaporizing asteroid', asteroid_counter, 'at', asteroid_coords)
            asteroid_counter += 1
            asteroids = asteroids[1:]
            if len(asteroids) > 0:
                new_list.append( (angle, direction, asteroids)  )
        asteroid_list = new_list
    

In [ ]:
vaporizeAsteroids(sorted_asteroids)

#### Test 2
In big example (with monitor station at (11, 13)):
* The 1st asteroid to be vaporized is at 11,12.
* The 2nd asteroid to be vaporized is at 12,1.
* The 3rd asteroid to be vaporized is at 12,2.
* The 10th asteroid to be vaporized is at 12,8.
* The 20th asteroid to be vaporized is at 16,0.
* The 50th asteroid to be vaporized is at 16,9.
* The 100th asteroid to be vaporized is at 10,16.
* The 199th asteroid to be vaporized is at 9,6.
* The 200th asteroid to be vaporized is at 8,2.
* The 201st asteroid to be vaporized is at 10,9.
* The 299th and final asteroid to be vaporized is at 11,1.

In [ ]:
_, all_asteroids_from_station, _, _ = probeAsteroid(big_example_int_map, 11, 13)
sorted_asteroids = sortAsteroidsForDestructionGroupedByDirection(all_asteroids_from_station)
vaporizeAsteroids(sorted_asteroids)

### Solution
Monitor station at (11, 19) with 253 visible asteroids

In [ ]:
_, all_asteroids_from_station, _, _ = probeAsteroid(int_map_data10, 11, 19)
sorted_asteroids = sortAsteroidsForDestructionGroupedByDirection(all_asteroids_from_station)
vaporizeAsteroids(sorted_asteroids)

Vaporizing asteroid 200 at (8, 15)

>>>Answer is 815